# Machine Learning with ToyotaHome/NVIDIA

## READ ME:
Run all cells in ascending order

### Setup/Imports

In [1]:
# Functions to use along widgets
from IPython.display import HTML, display, Markdown, Video
# Faciliate file selection
from tkinter import *
from tkinter import filedialog
# Widget Packages
import ipywidgets as widgets
# jupyter nbextension enable --py widgetsnbextension
# Used for local directory
import os

### Video Selector Functionality

In [2]:
path = os.getcwd() # print(os.path.basename(path)) # get the local directory
videoSelector = widgets.Button(description = "Select Video Preview")
videoClearer = widgets.Button(description = "Clear Video Preview")

#Create the videoplayer here
video_output = widgets.Output() # Customisation

def video_Select(b):
    root = Tk()
    root.filename =  filedialog.askopenfilename(
        initialdir = path,title = "Select file",filetypes = (("videos","*.mp4"),("all files","*.*")))
    FileName = os.path.basename(root.filename)
    # Get directory path
    cwd = os.getcwd()
    # Get file path
    AbsoluteFilePath = os.path.normpath(root.filename)
    # Replace file path's directory nonsense with the relative command
    FilePath = AbsoluteFilePath.replace(cwd,".")
    root.destroy() #Closes the Window
    with video_output:
        # Create Video File
        video_output.clear_output()
        html_video_code = '<video width="100%" height="100%" controls><source src="{filepath}" type="video/mp4"></video>'.format(filepath = FilePath)
        video_output.append_display_data(HTML(html_video_code))

def video_Clear(b):
    with video_output:
        video_output.clear_output()

#Link the button to the function
videoSelector.on_click(video_Select)
videoClearer.on_click(video_Clear)

### Shared Widgets

In [3]:
selected_Directory = widgets.Text(
    description = 'Directory',
    value = '',
    placeholder = 'Select Directory'
    )
current_Directory = widgets.Label()
tooltip_Dir = widgets.Label("Current Directory: ")
selectDir = widgets.Button(description = "Select Directory")
clearDir = widgets.Button(description = "Clear")
confirmDir = widgets.Button(description = "Confirm")

sharedWidgets = widgets.VBox([widgets.HBox([tooltip_Dir,current_Directory]),widgets.HBox([selectDir, confirmDir, clearDir])])

def directory_Select(b):
    root = Tk()
    root.directory =  filedialog.askdirectory (
        initialdir = path)
    selected_Directory.value = root.directory
    root.destroy() #Closes the Window\n",

def directory_Clear(b):
    current_Directory.value = ""

def directory_Print(b):
    current_Directory.value = selected_Directory.value
selectDir.on_click(directory_Select)
clearDir.on_click(directory_Clear)
confirmDir.on_click(directory_Print)

###  Toyota Widgets
These are the widgets required to run Toyota ML

In [4]:
Toyota = widgets.IntSlider(description = "Toyo")

### NVIDIA Widgets
These are the widgets required to run NVIDIA ML

In [5]:
Nvidia = widgets.IntSlider(description = "Nvidia")

In [6]:
# Pre-requisites
"""
List of Machine Learning stuff
Consider making it a text file to store/edit?
"""
ml_list = ['ToyotaHome', 'Nvidia']

# Widgets
mlSelectConfirm = widgets.Button(description="Confirm")
menu = widgets.Dropdown(
    options=ml_list,
    value=ml_list[0],
    description='ML Model:')

# Output
widgetset = widgets.Output()

# Function
def selectWidgetSet(b):
    with widgetset:
        widgetset.clear_output()
        if(menu.value == ml_list[0]): # if Toyota
            display(Toyota, sharedWidgets)
        elif(menu.value == ml_list[1]): # if Nvidia
            display(Nvidia, sharedWidgets)


mlSelectConfirm.on_click(selectWidgetSet)

### Other iPywidgets

#### Relevant links (in case I forget)
Output Widget
https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
<br>Widget List
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
<br>Linking Widgets
https://ipywidgets.readthedocs.io/en/7.6.2/examples/Widget%20Events.html
<br>Clear Output
https://stackoverflow.com/questions/54029719/jupyter-ipywidgets-how-to-clear-cell-output-before-re-printing-the-output
<br>Video Playback
https://towardsdatascience.com/video-streaming-in-the-jupyter-notebook-635bc5809e85

##### Markdown Syntax
https://www.markdownguide.org/basic-syntax

In [7]:
# calendar = widgets.DatePicker(
#     description='Select Date')

# checkbox = widgets.Checkbox(description='Check to invert',)

# text = widgets.Text(
#     value='None',
#     description='Current ML', )

# Linking Widgets
# l = widgets.link((slider1, 'value'), (slider2, 'value'))

# fileUpload = widgets.FileUpload(
#     accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
#     multiple=False  # True to accept multiple files upload else False
# )

In [8]:
FileName = r"./kh.mp4"
html_video_code = '<video width="100%" height="100%" controls><source src="{filepath}" type="video/mp4"></video>'.format(filepath = FileName)
def playVideo():
    HTML(html_video_code)
playVideo()
# from ipywidgets import Video, Audio
# video = Video.from_file(r'C:\Users\sheng\OneDrive - Singapore Institute Of Technology\Documents\league highlights\kh.mp4')
# video

In [9]:
# calendar = widgets.DatePicker(
#     description='Select Date')

# checkbox = widgets.Checkbox(description='Check to invert',)

# text = widgets.Text(
#     value='None',
#     description='Current ML', )

# Linking Widgets
# l = widgets.link((slider1, 'value'), (slider2, 'value'))

# fileUpload = widgets.FileUpload(
#     accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
#     multiple=False  # True to accept multiple files upload else False
# )

In [10]:
FileName = r"./kh.mp4"
html_video_code = '<video width="100%" height="100%" controls><source src="{filepath}" type="video/mp4"></video>'.format(filepath = FileName)
def playVideo():
    HTML(html_video_code)
playVideo()
# from ipywidgets import Video, Audio
# video = Video.from_file(r'C:\Users\sheng\OneDrive - Singapore Institute Of Technology\Documents\league highlights\kh.mp4')
# video

In [11]:
title = widgets.Label("Control Panel")
tooltip = widgets.Label("Supported Video Types: MP4, WebM, and OGG.")

mlBox = widgets.VBox([widgets.HBox([menu, mlSelectConfirm]),widgetset])
videoBox = widgets.VBox([widgets.HBox([videoSelector, videoClearer, tooltip]),video_output])
toolBox = widgets.VBox([title,mlBox,videoBox])

toolBox

### ML Widget Selector Functionality

# 1. Data Exploration

## 1.1 Select a video to playback

In [19]:
# @hidden_cell
## retrieve all videos to dropdown list
folder_files = os.listdir('data/rgbVideos') #You can also use full path.
print("This Folder contains {len_folder} file(s).".format(len_folder=len(folder_files)))
fileList=[]
for file in folder_files:
    fileList.append(file)


## dropdown list to select video to play
videoSelected = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    description="Video: ",
    disabled=False,
)
display(videoSelected)
#https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Button
videoButton = widgets.Button(
    description='Play video'
)
video_output = widgets.Output()
def explore_video(b):
    with video_output:
        # Create Video File
        video_output.clear_output()
        html_video_code = '<video width="80%" height="80%" controls><source src="./data/rgbVideos/{fileName}" type="video/mp4"></video>'.format(fileName = videoSelected.value)
        video_output.append_display_data(HTML(html_video_code))


videoButton.on_click(explore_video)

clearButton = widgets.Button(
    description='Clear Playback'
)

def clear_video(b):
    with video_output:
        video_output.clear_output()
        
# display(clearButton)
clearButton.on_click(clear_video)

## to align the buttons to 1 row
display(clearButton)
display(videoButton, video_output)

This Folder contains 536 file(s).


Dropdown(description='Video: ', options=('P02T01C06.mp4', 'P02T01C07.mp4', 'P02T02C03.mp4', 'P02T02C06.mp4', '…

Button(description='Clear Playback', style=ButtonStyle())

Button(description='Play video', style=ButtonStyle())

Output()

# 2. Inference Section

## Load a pretrained model

In [14]:
model_dir = './models/'
# listdir() returns a list containing the names of the entries in the directory given by path.
modelList = os.listdir("models")

selectModel = widgets.Dropdown(
    options = modelList,
    value = modelList[0],
    description = "Choose a pre-trained model",
    disabled= False
)

selectModel
# print(selectModel.value)

Dropdown(description='Choose a pre-trained model', options=('PDAN_TSU_RGB',), value='PDAN_TSU_RGB')

## Choose an input video from the TSU project

In [15]:
import os

## retrieve all TSU videos to dropdown list
folder_files = os.listdir('data/rgbVideos') 
print("This Folder contains {len_folder} file(s).".format(len_folder=len(folder_files)))
fileList=[]
for file in folder_files:
    fileList.append(file)

selected_input_video = widgets.Dropdown(
    options = fileList,
    value= fileList[0],
    description="Video: ",
    disabled=False,
)
display(selected_input_video)

select_button = widgets.Button(
    description='Select video'
)
display(select_button)


# dropdown selected value --> selected_input_video.value
# def get_selected_value(b):
#     print(selected_input_video.value)
# select_button.on_click(get_selected_value)

This Folder contains 536 file(s).


Dropdown(description='Video: ', options=('P02T01C06.mp4', 'P02T01C07.mp4', 'P02T02C03.mp4', 'P02T02C06.mp4', '…

Button(description='Select video', style=ButtonStyle())

## Inference Results

In [ ]:
##Install dependencies

!pip install -r requirements.txt

In [14]:
#see inference results in the form of output video with captions indicating detected activity in each video frame

pretrainModel = selectModel.value
videoName = selected_input_video.value

%run ./test.py -load-model "./"+pretrainModel


In [ ]:
## display video with captions

# 3. Training Section

## dropdown list to choose dataset for training

In [15]:
# dropdown list to choose dataset for training 
#"code here..."


## initialize model

In [16]:
# initialize model with a network architecture configured in a separate .py file
#"code here..."

## set name with text input 

In [17]:
# Specify a name for this new model using appropriate UI elements.
#"code here..."

## Set the batch_size and epochs

In [18]:
# Set the batch_size and epochs, with appropriate UI elements, prior to running a training sequence

## Run the training sequence, i.e., fit the model onto the dataset

In [19]:
# Run the training sequence, i.e., fit the model onto the dataset

## Visuals for progress of training

In [20]:
# See some visual elements to indicate the progress of the training in the notebook

## After training is complete, the trained model should be added to the list of pre-trained models that can be chosen in R3

In [21]:
# After training is complete, the trained model should be added to the list of pre-trained models that can be chosen in R3

# 4. Testing Section

## Choose a dataset subfolder, using appropriate UI elements, from the data folder to use for testing

In [22]:
# Choose a dataset subfolder, using appropriate UI elements, from the data folder to use for testing

## Load a pretrained model using an appropriate UI component

In [23]:
# Load a pretrained model using an appropriate UI component

## Run the testing sequence

In [24]:
# Run the testing sequence, i.e., perform inference on each data sample and accumulate some statistics
# (again check out the train.py file)

## See some visual elements to indicate the progress of testing in the notebook

In [25]:
# •	View some results 
# (e.g., Average Precision per activity class and mean Average Precision) 
# that allow for an assessment of how well the model performed

## Save the results to a results folder in the repo

In [26]:
# Save the results to a results folder in the repo

# 5. New Pipeline (NVIDIA STEP)

In [27]:
## STEP is just one other HOI project we suggest, but you are free to use any others. A good resource may be https://paperswithcode.com